# SEN163A - Fundamentals of Data Analytics
# Assignment 2 - Large-scale Internet Data Analysis
### Ir. Jacopo De Stefani - [J.deStefani@tudelft.nl](mailto:J.deStefani@tudelft.nl)
### Joao Pizani Flor, M.Sc. - [J.p.pizaniflor@tudelft.nl](mailto:J.p.pizaniflor@tudelft.nl)

### 05-03-2022
## Group 2
- Emmanuel M Boateng - '5617642'
- Joost Oortwijn - '4593472'
- Philip Busscher - ''4611993''
- Floris Kool - ''4975243''


# Introduction
short description of the assignment

# 1. Dataset description

Short description of the 4 datasets used

## 1.1 Opening the data

In [ ]:
import pickle
import time
import bz2
import os
import sys
import json
import pandas
import io
import datetime
import pandas


In [ ]:
#AS Dataset

AS_Filename = 'data/AS_dataset.pkl'

with open(AS_Filename, 'rb') as file:
    
    AS_df = pickle.load(file)

AS_df.head()


In [ ]:
#Probe dataset

Probe_Filename = 'data/probe_dataset.pkl'

with open(Probe_Filename, 'rb') as file:
    
    P_df = pickle.load(file)

P_df.head()

In [ ]:
#Ripe dataset 


#Method needs to be decided

In [ ]:

merged_df = P_df.merge(AS_df)

merged_df.insert(2, 'AS', merged_df['ASN'].str.replace('AS',''))
merged_df['AS'] = pandas.to_numeric(merged_df['AS'])
merged_df['prb_id'] = pandas.to_numeric(merged_df['prb_id'])


merged_df = merged_df.sort_values('AS')
merged_df.head(3)

print(merged_df.type.value_counts().hosting)
#print(merged_df.loc[(merged_df['type']=='') | (merged_df['Country']=="")])

#merged_df.info('NumIPs')
#& (df['FT_Team'].str.startswith('S')),['Name','FT_Team']]
EU_Countries = ["AT","BE","HR","CY","CZ","DK","EE","FI","FR","GR","DE","HU",
                "IE","IT","LV","LT","LU","MT","NL","PL","PT","RO","SK","SI",
                "ES","SE"]

EU_data = merged_df[merged_df['Country'].isin(EU_Countries)]

the first three ASNs

In [ ]:
merged_df.head(3)

the last three ASNs

In [ ]:
merged_df.tail(3)

In [ ]:
#IP 2 Location dataset

IP_Filename = "data/IP2LOCATION-LITE-DB1.CSV"

ipv4_df = pandas.read_csv(IP_Filename)

ipv4_df.head()

More detailed description of data if needed (Can also be after opening each dataset)

## 1.2 Limitations in data

Evaluate if there are limitations in the provided datasets (AS and probe data set). If you find limitations, describe these and conjecture possible reasons, supported with data.

...

In [ ]:
#Code needed to prove limitations

In [ ]:
bz2Filename = 'data/ping-2022-03-01T2300.bz2'
bz2File     = bz2.open(bz2Filename, 'rt') 
firstLine = bz2File.readline();
firstLine_sizeInBytes   = sys.getsizeof(firstLine)
print("size of line 1",firstLine_sizeInBytes)

Some list of limitations in text

# 2 Analysis

Short description of what is going to be analyzed

## 2.1 AS

With the AS and probe data set, find the number m of AS’s that can be used for hosting in the EU
and have probes in the RIPE data set. Sort the ASN’s in ascending order and include the first and last
three in your report (number, name and country).


In [ ]:
#Upper part should be removed because run in part 1
import pickle
import time
import bz2
import os
import sys
import json
import pandas
import io
import datetime

with open('data/AS_dataset.pkl', 'rb') as file:
    AS_df = pickle.load(file)
    
with open('data/probe_dataset.pkl', 'rb') as file:    
    P_df = pickle.load(file)
    
decomFilename = 'data/ping-2022-03-01T2300.bz2'
#decomFile     = bz2.open(decomFilename, 'rt')   
merged_df = P_df.merge(AS_df)

merged_df.insert(2, 'AS', merged_df['ASN'].str.replace('AS',''))
merged_df['AS'] = pandas.to_numeric(merged_df['AS'])
merged_df['prb_id'] = pandas.to_numeric(merged_df['prb_id'])


merged_df = merged_df.sort_values('AS')
merged_df.head(3)

#print(merged_df.type.value_counts().hosting)
#print(merged_df.loc[(merged_df['type']=='') | (merged_df['Country']=="")])

#merged_df.info('NumIPs')
#& (df['FT_Team'].str.startswith('S')),['Name','FT_Team']]
EU_Countries = ["AT","BE","HR","CY","CZ","DK","EE","FI","FR","GR","DE","HU",
                "IE","IT","LV","LT","LU","MT","NL","PL","PT","RO","SK","SI",
                "ES","SE"]

EU_data = merged_df[merged_df['Country'].isin(EU_Countries)]
EU_Hosting = EU_data[EU_data['type'] == 'hosting']


rmv_list =['result','fw','mver','lts','af','proto','size','dup','rcvd','sent','min', 'ttl','max','msm_id','msm_name','step']
#Create list of all probes that are in the RIPE dataset

t0 = time.time()
time.sleep(0.000001)
with open(decomFilename, 'rb') as fi:
    decomp = bz2.BZ2Decompressor()
    residue = b''
    total_lines = 0
    m = 0
    checked = []
    for data in iter(lambda: fi.read(100 * 1024), b''):
        raw = residue + decomp.decompress(data) # process the raw data and  concatenate residual of the previous block to the beginning of the current raw data block
        residue = b''
        # process_data(current_block) => do the processing of the current data block
        current_block = raw.split(b'\n')
        if raw[-1] != b'\n':
            residue = current_block.pop() # last line could be incomplete
            
        for items in current_block:
            df_dict = json.loads(items.decode('utf-8'))
            if (df_dict['prb_id'] in EU_Hosting["prb_id"]) and (df_dict['af'] == 4):
                if df_dict['prb_id'] not in checked:
                    m +=1
                    checked.append(df_dict['prb_id'])
                    #print(checked)
        total_lines += len(current_block)
    total_lines += 1
    #print('Final: %i lines/sec' % (total_lines / (time.time() - t0)))
#print(data)
#print("*****")
#print(current_block)
print("Total processing time: ",(time.time() - t0))
print("total lines :", total_lines)
print("Total number of EU hosting probes in RIPE is %i" %(m))
fi.close()
#######-------good code

In [ ]:
import pickle
import time
import bz2
import os
import sys
import json
import pandas
import io
import datetime
import socket
import struct

def ip2int(addr):
    return struct.unpack("!I", socket.inet_aton(addr))[0]

with open('data/AS_dataset.pkl', 'rb') as file:
    AS_df = pickle.load(file)
    
with open('data/probe_dataset.pkl', 'rb') as file:    
    P_df = pickle.load(file)
    
decomFilename = 'data/ping-2022-03-01T2300.bz2'
#decomFile     = bz2.open(decomFilename, 'rt')   
merged_df = P_df.merge(AS_df)

ipv4_df = pandas.read_csv("data/IP2LOCATION-LITE-DB1.CSV")
ipv4_df.rename(columns = {'0':'ip_from', '16777215':'ip_to',
                              '-':'country_code','-.1':'country_name'}, inplace = True)


EU_Countries = ["AT","BE","HR","CY","CZ","DK","EE","FI","FR","GR","DE","HU",
                "IE","IT","LV","LT","LU","MT","NL","PL","PT","RO","SK","SI",
                "ES","SE"]

EU_data = merged_df[merged_df['Country'].isin(EU_Countries)]
EU_Hosting = EU_data[EU_data['type'] == 'hosting']



merged_df.insert(2, 'AS', merged_df['ASN'].str.replace('AS',''))
merged_df['AS'] = pandas.to_numeric(merged_df['AS'])
merged_df['prb_id'] = pandas.to_numeric(merged_df['prb_id'])


merged_df_sorted = merged_df.sort_values('AS')
df_HostingAS = merged_df[merged_df['type'] == 'hosting']

ipv4_df.head()
tpl = ipv4_df.loc[:, 'ip_from':'ip_to'].apply(tuple, 1).tolist()
idx = pandas.IntervalIndex.from_tuples(tpl, 'both')

t0 = time.time()
time.sleep(0.000001)
with open(decomFilename, 'rb') as file:
    decomp = bz2.BZ2Decompressor()
    residue = b''
    total_lines = 0
    m = 0
    checked = []
    #102400 Bytes = 102.4 KB (in decimal)
    #102400 Bytes = 100 KB (in binary)
    #Iterate over RIPE data in  100 KB chunks 
    for data in iter(lambda: file.read(100 * 1024), b''):
        # process the raw data and  concatenate residual of the previous block 
        #to the beginning of the current raw data block
        raw = residue + decomp.decompress(data) 
        residue = b''
        ## process_data(current_block) => do the processing of the 
        ##current data block
        current_block = raw.split(b'\n')
        if raw[-1] != b'\n':
            residue = current_block.pop() # last line could be incomplete
        ##Process all data in the current block to check    
        for items in current_block:
            df_dict = json.loads(items.decode('utf-8'))
            if ('dst_addr' in df_dict) and (df_dict['af'] == 4):# and (ip2int(df_dict['dst_addr'])>0:
                ##convert to interger
                df_ip = ip2int(df_dict['dst_addr'])
                #print(df_ip)
                if df_ip > 0: # certain lines have 0.0.0.0 IP
                    loc = idx.get_loc(df_ip)
                    if ((ipv4_df.loc[loc,'country_code'] in EU_Countries) and (df_dict['prb_id'] not in checked)):
                        #if len(EU_Hosting[EU_Hosting['prb_id'] == df_dict['prb_id']])!=0:
                            #print(df_HostingAS[df_HostingAS['prb_id'] == df_dict['prb_id']])
                        m +=1 ## increment count
                       ##create a list of probes that could be used later                     
                        checked.append(df_dict['prb_id']) 
        total_lines += len(current_block)
    total_lines += 1

print("Total processing time: ",(time.time() - t0))
print("Total number of probe entries with hosting type AS and EU target in RIPE is %i" %(m))
fi.close()


In [10]:
import pickle
import time
import bz2
import os
import sys
import json
import pandas
import io
import datetime
import socket
import struct

def ip2int(addr):
    return struct.unpack("!I", socket.inet_aton(addr))[0]

with open('data/AS_dataset.pkl', 'rb') as file:
    AS_df = pickle.load(file)
    
with open('data/probe_dataset.pkl', 'rb') as file:    
    P_df = pickle.load(file)
    
decomFilename = 'data/ping-2022-03-01T2300.bz2'
#decomFile     = bz2.open(decomFilename, 'rt')   
merged_df = P_df.merge(AS_df)

ipv4_df = pandas.read_csv("data/IP2LOCATION-LITE-DB1.CSV")
ipv4_df.rename(columns = {'0':'ip_from', '16777215':'ip_to',
                              '-':'country_code','-.1':'country_name'}, inplace = True)


EU_Countries = ["AT","BE","HR","CY","CZ","DK","EE","FI","FR","GR","DE","HU",
                "IE","IT","LV","LT","LU","MT","NL","PL","PT","RO","SK","SI",
                "ES","SE"]

EU_data = merged_df[merged_df['Country'].isin(EU_Countries)]
EU_Hosting = EU_data[EU_data['type'] == 'hosting']



merged_df.insert(2, 'AS', merged_df['ASN'].str.replace('AS',''))
merged_df['AS'] = pandas.to_numeric(merged_df['AS'])
merged_df['prb_id'] = pandas.to_numeric(merged_df['prb_id'])


merged_df_sorted = merged_df.sort_values('AS')
df_HostingAS = merged_df[merged_df['type'] == 'hosting']
EU_Hosting.head()

,prb_id,ASN,Country,Name,NumIPs,type
1637,57,AS20621,DE,PlusServer GmbH,"10,240",hosting
2451,92,AS12859,NL,BIT BV,"66,816",hosting
2452,6234,AS12859,NL,BIT BV,"66,816",hosting
2453,6237,AS12859,NL,BIT BV,"66,816",hosting
2454,32445,AS12859,NL,BIT BV,"66,816",hosting


In [ ]:
ipv4_df.head()


In [ ]:
####-----------------------
#### Code to exclusively read the whole ripe file
#####-----------------------
import pickle
import time
import bz2
import os
import sys
import json
import pandas
import io
import datetime

decomFilename = 'data/ping-2022-03-01T2300.bz2'


t0 = time.time()
time.sleep(0.000001)
with open(decomFilename, 'rb') as fi:
    decomp = bz2.BZ2Decompressor()
    residue = b''
    total_lines = 0
    for data in iter(lambda: fi.read(100 * 1024), b''):
        raw = residue + decomp.decompress(data) # process the raw data and  concatenate residual of the previous block to the beginning of the current raw data block
        residue = b''
        # process_data(current_block) => do the processing of the current data block
        current_block = raw.split(b'\n')
        if raw[-1] != b'\n':
            residue = current_block.pop() # last line could be incomplete
        total_lines += len(current_block)
    total_lines += 1
    #print('Final: %i lines/sec' % (total_lines / (time.time() - t0)))
#print(data)
#print("*****")
#print(current_block)
print("Total processing time: ",(time.time() - t0))
print("total lines :", total_lines)
fi.close()



In [ ]:
data =raw.splitlines()

rmv_list =['result','fw','mver','ttr','lts','af','proto','size','dup','rcvd','sent','min', 'ttl','max','msm_id','msm_name','step']
count = 0
RIPE_df = pandas.DataFrame()  
for items in data:
    count = count +1
    df_dict = json.loads(items.decode('utf-8'))
    for items in rmv_list:
        if items in df_dict:
            del df_dict[items]
            df = pandas.DataFrame([df_dict], columns=df_dict.keys())
    RIPE_df = pandas.concat([RIPE_df, df], axis =0).reset_index(drop=True)


In [ ]:
RIPE_df.tail()
#print(RIPE_df.groupby('prb_id').count())
print("Number of probes in Ripe:",RIPE_df['prb_id'].value_counts().sum())
print("Number of probes in Probe:", P_df['prb_id'].value_counts().sum())



In [ ]:
for line in decomFile:
    jsonline = json.loads(line)
    index = index +1
    d = datetime.datetime.utcfromtimestamp(jsonline["timestamp"])
    if d.minute <= 30:
        if jsonline["prb_id"] not in RIPEProbes:
            for items in rmv_list:
                if items in jsonline:
                    jsonline.pop(items)
            RIPEProbes.append(jsonline)
    if index == nrOfLines_half:
        break
                          
    #Read only first 1m lines
    #index = index + 1
    #if index > 200:#0000:
    #    break
                  
print("Probes in first 1m lines of RIPE Dataset: " +str(len(RIPEProbes)))            
ripe_df = pandas.DataFrame(RIPEProbes)
ripe_df.head()

In [ ]:
index = 0
ProbeASNs = []

#Create list of all probes in both RIPE and probe datasets
#Saves only the ASNs as these are used later
#Probe IDs no longer used after this point
for probe in P_df["prb_id"]:
    if probe in RIPEProbes:
        ProbeASNs.append(P_df["ASN"][index])
        
    index = index + 1
    
print("Probes in both RIPE and probe dataset: " + str(len(ProbeASNs)))    

In [ ]:
HostProbeASNs = []

index = 0

for ASN in AS_df["ASN"]:
    
    if ASN in ProbeASNs:
        if AS_df["type"][index] == "hosting":
            HostProbeASNs.append(ASN)            
    index = index + 1    
    
print("Amount of probes with an ASN with type hosting: " + str(len(HostProbeASNs))) 

HostProbeASNs.sort()

Description of results

## 2.2 Hosting location
For a single hour in the RIPE data set: find all valid entries where the probe has hosting type AS and
the target IPv4 is from an EU country. Implement this in an efficient way.

In [ ]:
HostProbes = []

for ProbeASN in HostProbeASNs:
    
    index = 0
    for ASN in P_df["ASN"]:
        if ASN == ProbeASN:
            HostProbes.append(P_df["prb_id"][index])
            break
        index = index + 1
        
print(len(HostProbes))

In [ ]:
decomFilename = 'C:/Users/Kooltje/Downloads/FoDa Data/ping-2022-03-01T2300'
decomFile     = open(decomFilename, 'rt')   

HostIPs = []
index = 0

for line in decomFile:
    jsonline = json.loads(line)
    
    
    if jsonline["prb_id"] in HostProbes:
        try:
            #Check for duplicates
            if jsonline["dst_addr"] not in HostIPs:
                #Check if IP is of type 4
                if jsonline["af"] == 4:
                    HostIPs.append(jsonline["dst_addr"])
        except KeyError as err:
            pass
    
    #Read only first 1m lines
    index = index + 1
    if index > 1000000:
        break
                              
print("Amount of IPs in the RIPE data connected to an AS of type Hosting: " + str(len(HostIPs)))

decomFile.close()

In [ ]:
#Emm

decomFilename = 'data/ping-2022-03-01T2300.bz2'
decomFile     = open(decomFilename, 'rb') 
bz2File     = bz2.open(decomFilename, 'rb') 

firstLine = decomFile.readline()

firstLine_sizeInBytes   = sys.getsizeof(firstLine) 
decomFile_sizeInBytes   = os.stat(decomFilename).st_size
nrOfLines               = round(decomFile_sizeInBytes/firstLine_sizeInBytes)
print("\nEstimated nr of lines = " + str(nrOfLines))


count = 0;
st    = time.time()
for line in bz2File:
    count = count + 1
    if count>100000: 
        break
    
#print time and estimate total time            
dur         = round(time.time() - st,2)
estTotTime  = round( (dur/100000)*nrOfLines )
print("\nDecompressed file:" )
print("Loading 100k lines took: "  + str(dur) + " seconds")
print("Estimated loading time of entire decompression file: "  + str(estTotTime) + " seconds" )

#---------------------
#Do needed conversion
#---------------------

print('-------')
datarow = []
counter = 0
encoding = 'utf-8'
keys = ['af','avg', 'dst_addr','dst_name','from','prb_id','src_addr','timestamp','type']
st    = time.time()

for line in bz2File:
    counter = counter +1
    data = 
    rmv_list =['result','fw','mver','lts','af','proto','size','dup','rcvd','sent','min', 'ttl','max','msm_id','msm_name','step']
    print(data)
    for items in rmv_list:
        if items in data:
            data.pop(items)
    datarow.append(data)
    if counter == 1000000:
        break
    
dur         = round(time.time() - st,2)
#estTotTime  = round( (dur/100000)*nrOfLines )
#print("\nDecompressed file:" )
print("\nLoading lines took: "  + str(dur) + " seconds")
#print("\nEstimated loading time of entire decompression file: "  + str(estTotTime) + " seconds" )
        
df = pandas.DataFrame(datarow)
decomFile.close()


In [ ]:
for line in decomFile:
    data = json.loads(line)
    print(type(data))
    res_dict = data.pop('result')
    datarow.append(res_dict)
    #counter = counter +1
    #if counter == 100000:
    #    print(line)
     #   break
        
        for line in decomFile:
    counter = counter +1
    if counter == 1:
        print(line)
        #dataform = str(response_json).strip("'<>() ").replace('\'', '\"')
        data = line.decode(encoding, errors= 'ignore')
        firstLineJson = json.loads(data)
        print(json.dumps(firstLineJson, sort_keys=True, indent=4))
        break

for line in bz2File:
    counter = counter +1
    data = json.loads(line)
    rmv_list =['result','fw','mver','lts','af','proto','size','dup','rcvd','sent','min', 'ttl','max','msm_id','msm_name','step']
    for items in rmv_list:
        if items in data:
            data.pop(items)
    datarow.append(data)
    #if counter == 1:
    #    print(data)
    #    break
    
dur         = round(time.time() - st,2)
#estTotTime  = round( (dur/100000)*nrOfLines )
#print("\nDecompressed file:" )
print("\nLoading lines took: "  + str(dur) + " seconds")
#print("\nEstimated loading time of entire decompression file: "  + str(estTotTime) + " seconds" )
        
df = pandas.DataFrame(datarow)
decomFile.close()

        

In [ ]:
df.shape

In [ ]:
sdata = b'{"fw":5020,"mver":"2.2.1","lts":-1,"dst_name":"192.5.5.241","af":4,"dst_addr":"192.5.5.241","src_addr":"172.17.0.2","proto":"ICMP","ttl":58,"size":32,"result":[{"rtt":13.444936},{"rtt":3.762035},{"rtt":3.557907}],"dup":0,"rcvd":3,"sent":3,"min":3.557907,"max":13.444936,"avg":6.921626,"msm_id":1004,"prb_id":1001660,"timestamp":1646176375,"msm_name":"Ping","from":"83.243.73.91","type":"ping","step":240}\n'

dict_str = sdata.decode("UTF-8")
my_data = ast.literal_eval(dict_str)
type(my_data)
print(my_data)

In [ ]:

# Driver code
dict_1 = {'John': 15, 'Rick': 10, 'Misa' : 12 }
dict_2 = {'John': 18,'Rick': 20,'Misa' : 16 }

case_list = [dict_1]

case_list.append(dict_2)
print(case_list)

df1 = pandas.DataFrame(case_list)
df1.head()


In [ ]:
print(df.shape)
df.head()

In [ ]:
#Converting the xxx.xxx.xxx.xxx format of the host IPs to integer
#Needed for when comparing to the IPv4 dataset

HostIPs_Integer = []


for IPString in HostIPs:
    IP_Splitstring = IPString.split(".") 
 
    HostIPs_Integer.append(int(IP_Splitstring[0]) * 16581375 + int(IP_Splitstring[1]) * 65025 + int(IP_Splitstring[2]) * 255 + int(IP_Splitstring[3]))  

print(len(HostIPs_Integer))



In [ ]:
#List of country codes that are an EU member

EU_Countries = ["AT",
    "BE",
    "HR",
    "CY",
    "CZ",
    "DK",
    "EE",
    "FI",
    "FR",
    "GR",
    "DE",
    "HU",
    "IE",
    "IT",
    "LV",
    "LT",
    "LU",
    "MT",
    "NL",
    "PL",
    "PT",
    "RO",
    "SK",
    "SI",
    "ES",
    "SE"]

In [ ]:
#Compare IPv4 with HostIPs_Integer

ipv4_df = pandas.read_csv("data/IP2LOCATION-LITE-DB1.CSV")
ipv4_df.rename(columns = {'0':'ip_from', '16777215':'ip_to',
                              '-':'country_code','-.1':'country_name'}, inplace = True)

ipv4_df.head()

HostIPs_EU = []
index = 0
ipv4index = 0

#Sorting the IP list so we can check from low to high IPs
HostIPs_Integer.sort

for IP_to in ipv4_df["ip_to"]:
    
    while HostIPs_Integer[index] < IP_to:
        if ipv4_df["country_code"][ipv4index] in EU_Countries:
            HostIPs_EU.append(HostIPs_Integer[index])
            print(HostIPs_Integer[index])
            print(ipv4_df["country_code"][ipv4index])
        index = index + 1
        if index >= len(HostIPs_Integer):
            break;
    ipv4index = ipv4index + 1  
    if index >= len(HostIPs_Integer):
        break;
                  

print(len(HostIPs_EU))


In [ ]:
ipv4_df.head()

Description of results

## 2.3 Latency
Move from using only an hour to the full day. It is advisable to store the raw results of each file. Then,
using all processed files, calculate the average latency’s for each country-AS combination and store
the results into one $n_{countries}$ × $m$ matrix. If we could place one server in each country, what would the
minimum average latency be for each country? (include in your report)


In [ ]:
#Code...
#Create a list of all the datasets to loop through

l =pandas.date_range('2022-03-01', periods=24, freq='60min').strftime('data/ping-%Y-%m-%dT%H%M.bz2').tolist()
print(l)


d = {}
for dataset in l:
    decomFilename = dataset
    tstamp = str(decomFilename.strip('.bz2')[-5:])
    
    with open(decomFilename, 'rb') as fi:
        decomp = bz2.BZ2Decompressor()
        residue = b''
        total_lines = 0
    for data in iter(lambda: fi.read(100 * 1024), b''):
        raw = residue + decomp.decompress(data) # process the raw data and  concatenate residual of the previous block to the beginning of the current raw data block
        residue = b''
        # process_data(current_block) => do the processing of the current data block
        current_block = raw.split(b'\n')
        if raw[-1] != b'\n':
            residue = current_block.pop() # last line could be incomplete
        for items in current_block:
            df_dict = json.loads(items.decode('utf-8'))
            d[tstamp] = d[tstamp].append(df_dict)
        total_lines += len(current_block)
    total_lines += 1
    
print("Total processing time: ",(time.time() - t0))
print("total lines :", total_lines)
fi.close()
    
    

In [ ]:
l =pandas.date_range('2022-03-01', periods=24, freq='60min').strftime('data/ping-%Y-%m-%dT%H%M.bz2').tolist()
#print(l)

d = {}
for dataset in l:
    decomFilename = dataset
    tstamp = str(decomFilename.strip('.bz2')[-5:])
    print(tstamp)

    with open(decomFilename, 'rb') as fi:
        decomp = bz2.BZ2Decompressor()
        residue = b''
        total_lines = 0
        d[tstamp] = []
        for data in iter(lambda: fi.read(100 * 1024), b''):
            raw = residue + decomp.decompress(data) # process the raw data and  concatenate residual of the previous block to the beginning of the current raw data block
            residue = b''
            # process_data(current_block) => do the processing of the current data block
            current_block = raw.split(b'\n')
            if raw[-1] != b'\n':
                residue = current_block.pop() # last line could be incomplete
            for items in current_block:
                df_dict = json.loads(items.decode('utf-8'))
                if (df_dict['af'] == 4):
                    res = {key: df_dict[key] for key in df_dict.keys()
                               & {'af', 'avg', 'from','dst_addr','prb_id'}}
                    
                    d[tstamp].append(res)
                    
            #print(len(d))
            #print(sys.getsizeof(d))
            #print(d[tstamp])
            #print("length is ", len(d[tstamp]))
            #print("size is ",sys.getsizeof(d[tstamp]))
break


In [ ]:
dictionary1 = {  'Pen': 5, 'Pencil': 4, 'Chocolate' : 15 }
dictionary2 = {'Apple': 25,'Ball': 10,'Doll' : 20 }
dictionary1.update(dictionary2)
print(dictionary1)

Description of results

## 2.4 Optimal server locations
Since we are only allowed to place four servers, determine the best four datacenters based on the total
latency for all countries. Report your findings and your procedure to obtain them. Also include the
average latency for each country.


In [ ]:
#Code...

0Description of results

# Conclusions

... 
add code if needed

In [ ]:
index = 0
ProbeASNs = []

#Create list of all probes in both RIPE and probe datasets
#Saves only the ASNs as these are used later
#Probe IDs no longer used after this point
for probe in P_df["prb_id"]:
    if probe in RIPEProbes:
        ProbeASNs.append(P_df["ASN"][index])
        
    index = index + 1
    
print("Probes in both RIPE and probe dataset: " + str(len(ProbeASNs)))    

In [ ]:
HostProbes = []

index = 0

print(len(HostProbeASNs))

for ASN in P_df["ASN"]:
    if ASN in HostProbeASNs:
        if P_df["prb_id"][index] not in HostProbes:
            HostProbes.append(P_df["prb_id"][index])
    index = index + 1

print(len(HostProbes))    

In [ ]:
0#Random stuff I didn't want to throw away yet
#0Code for finding all host probes from EU in the dataset of one hour
import time
import bz2
import os
import sys
import json

# open decompressed file
decomFilename = 'C:/Users/Kooltje/Downloads/FoDa Data/ping-2022-03-01T2300'
decomFile     = open(decomFilename, 'rt') 

#read first line and print
#firstLine = decomFile.readline();
#print(firstLine)

#the line appears to be json-formatted: pretty print json
#firstLineJson = json.loads(firstLine)

#read all lines of first file
count = 0
st    = time.time()
for line in decomFile:
    jsonline = json.loads(line)
    #print(json.dumps(jsonline, sort_keys=True, indent=4))
    count = count + 1
    if count > 10000: 
          break

#print the last line
print(json.dumps(jsonline, sort_keys=True, indent=4))

#print the read duration
dur         = round(time.time() - st,2)
print("Loading took: " + str(dur) + " seconds")
print("The file had " + str(count) + "lines")

#finally close decomFile
decomFile.close()